In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib as mpl

import numpy as np
import pandas as pd

from matplotlib import font_manager, rc

import folium
import branca
from shapely.geometry.point import Point

from utils import round, cal_area, generate_candidate_sites
from mclp_function import mclp

In [ ]:
postbox = pd.read_csv('./data/org/서울시 안심택배함 설치 장소.csv', encoding='cp949')
postbox = postbox[postbox['자치구'] == '강남구'].reset_index()

police = pd.read_csv('./data/police.csv')
small_police = pd.read_csv('./data/police_small.csv')
metro = pd.read_csv('./data/metro.csv')
department = pd.read_csv('./data/department.csv')

In [ ]:
radius = 500
y_u = round(lat='WGS Y 좌표', lon='WGS X 좌표', dataset=postbox, r=radius)  # 안심택배함
police_u = round(lat='lat', lon='lon', dataset=police, r=radius)  # 지구대
small_police_u = round(lat='lat', lon='lon', dataset=small_police)  # 파출소
metro_u = round(lat='위도', lon='경도', dataset=metro)  # 지하철
department_u = round(lat='lat', lon='lon', dataset=department)  # 파출소

In [ ]:
lat_c, lon_c = '37.498095', '127.027610'

m = folium.Map([lat_c, lon_c], zoom_start = 13)
for lat, lon in zip(postbox['WGS Y 좌표'], postbox['WGS X 좌표']):
    folium.Circle(location = [lat, lon], radius = radius, color='black', fill_color = 'purple', weight=1).add_to(m)
for lat, lon in zip(police['lat'], police['lon']):
    folium.Circle(location = [lat, lon], radius = radius, color='black', fill_color = 'red', weight=1).add_to(m)
for lat, lon in zip(small_police['lat'], small_police['lon']):
    folium.Circle(location = [lat, lon], radius = 250, color='black', fill_color = 'red', weight=1).add_to(m)
for lat, lon in zip(metro['위도'], metro['경도']):
    folium.Circle(location = [lat, lon], radius = 250, color='black', fill_color = 'green', weight=1).add_to(m)
for lat, lon in zip(department['lat'], department['lon']):
    folium.Circle(location = [lat, lon], radius = 250, color='black', fill_color = 'green', weight=1).add_to(m)
m #m.save('./map/visuallize_gu.html')

In [ ]:
nlsp = gpd.read_file('./data/org/nlsp_021001001.shp',
                     encoding='utf-8').to_crs(epsg=4326)

m2 = folium.Map([lat_c, lon_c], zoom_start = 13)
for _, row in nlsp.iterrows():
    value = 1 - cal_area(row['geometry'], u)
    sim_geo = gpd.GeoSeries(row['geometry'])
    geo_j = sim_geo.to_json()
    color = plt.cm.OrRd(value)
    color = mpl.colors.to_hex(color)
    
    geo_j = folium.GeoJson(data=geo_j,
                        style_function=lambda feature, color=color: {
                                                                        'fillColor': color,
                                                                        'color':"black",
                                                                        'weight': 2,
                                                                        'dashArray': '5, 5',
                                                                        'fillOpacity': 0.8,
                                                                    })


    folium.Popup(f'{value:.3f}').add_to(geo_j)
    geo_j.add_to(m2)

colormap = branca.colormap.linear.OrRd_06.scale(0, 1)
colormap = colormap.to_step(index=np.linspace(0, 1, 1000))
colormap.caption = 'Area Ratio NOT Covered by postbox_area'
colormap.add_to(m2)
m2 # m2.save('./map/demand_safe_delivery_box_area.html')

**$w_i = lbl \times AreaRatio \times restaurant \times bar \times academy $**

- $w_i$ : 수요량
- $lbl$ : 주거 인구 비례
- $police$ : 지구대
- $smallpolice$ : 파출소
- $metro$ : 지하철
- $department$ : 백화점
- $AreaRatio$ : 커버되지않은 면적 비율 (0~1)

In [ ]:
m2_nlsp = nlsp.replace(to_replace='N/A', value=0)
m2_nlsp = m2_nlsp.astype({'lbl': float})

In [ ]:
m3 = folium.Map([lat_c, lon_c], zoom_start=13)
values = []
cnt = 0
for _, row in m2_nlsp.iterrows():
    value = (1-cal_area(row['geometry'], y_u)) * (row['lbl']) * (1 - 0.5 * cal_area(row['geometry'], police_u)) * (1 - 0.3 * cal_area(
        row['geometry'], small_police_u)) * (1 + 0.8 * cal_area(row['geometry'], metro_u)) * (1 + 0.6 * cal_area(row['geometry'], department_u))
    if value >= 0.0005:
        cnt += 1
    else:
        value = 0
    values.append(value)
max_value = max(values)

for i, row in m2_nlsp.iterrows():
    value = values[i]
    sim_geo = gpd.GeoSeries(row['geometry'])
    geo_j = sim_geo.to_json()

    color = plt.cm.Reds(value/max_value)
    color = mpl.colors.to_hex(color)

    if value == 0:
        geo_j = folium.GeoJson(data=geo_j,
                               style_function=lambda feature, color=color: {
                                   'fillColor': color,
                                   'color': "black",
                                   'weight': 0.5,
                                   'dashArray': '5, 5',
                                   'fillOpacity': 0,
                               })
    else:
        geo_j = folium.GeoJson(data=geo_j,
                               style_function=lambda feature, color=color: {
                                   'fillColor': color,
                                   'color': "black",
                                   'weight': 0.5,
                                   'dashArray': '5, 5',
                                   'fillOpacity': 0.8,
                               })

    folium.Popup(f'{value:.3f}').add_to(geo_j)
    geo_j.add_to(m3)

colormap = branca.colormap.linear.Reds_05.scale(0, max_value)
colormap = colormap.to_step(index=np.linspace(0, max_value, 1000))
colormap.caption = 'lbl × value not covered by safe_delivery_box'
colormap.add_to(m3)

m3 # m3.save('./map/value_area_safe_delivery_box.html')


In [ ]:
print(f'max_value : {max_value:.4f}')
print(f'N_Grid(value>0) : {cnt}, N_Grid(All) : {len(m2_nlsp)}')
print(f'Grid_Ratio : {cnt/len(m2_nlsp):.4f}')

In [ ]:
m2_nlsp['weight'] = values
m2_nlsp['geo'] = nlsp['geometry'].to_crs(epsg=5179)

M = cnt
K = 0
l = len(m2_nlsp)

new_df = pd.DataFrame(data={'K': [0], 'max_value': [np.round(max_value, 4)], 'mean_value': [np.mean(m2_nlsp['weight'])],
                            'N_Grid(value>0)': cnt, 'Grid_Ratio': np.round(cnt/l, 4)})
new_df['K'] = new_df['K'].astype(int)
while True:
    K += 1
    opt_sites_org, f = mclp(m2_nlsp, K, M, radius)

    opt_df = pd.DataFrame(opt_sites_org, columns=['lon', 'lat'])
    opt_df['geom'] = opt_df.apply(
        lambda row: Point(row['lon'], row['lat']), axis=1)
    gdf = gpd.GeoDataFrame(opt_df, geometry='geom', crs='epsg:5179')
    gdf = gdf.to_crs('epsg:4326')
    gdf['lon'] = gdf['geom'].apply(lambda p: p.x)
    gdf['lat'] = gdf['geom'].apply(lambda p: p.y)
    lats = gdf['lat']
    lons = gdf['lon']
    opt_df = gpd.GeoDataFrame(opt_df, geometry='geom', crs='epsg:5179')
    opt_df['geometry'] = opt_df.geom.buffer(radius)
    gdf = opt_df.to_crs('epsg:4326')

    u_tmp = y_u
    for c in gdf['circle']:
        u_tmp = u_tmp.union(c)

    values = []
    for _, row in m2_nlsp.iterrows():
        value = (1 - cal_area(row['geometry'], u_tmp)) * row['weight']
        if value < 0.1:
            value = 0
        values.append(value)
    max_value = max(values)

    tmp_df = pd.DataFrame({'K': [int(K)], 'max_value': [np.round(max_value, 4)], 'mean_value': [np.mean(values)],
                           'N_Grid(value>0)': [cnt-f], 'Grid_Ratio': [np.round((cnt-f)/l, 4)]},
                          index=[K])
    new_df = pd.concat([new_df, tmp_df])

    if f == M:
        break

print(f'최소 {K}개의 안심박스로 모든 취약 지점을 커버할 수 있다.')
new_df

In [ ]:
m4 = folium.Map([lat_c, lon_c], zoom_start=13)
values = m2_nlsp['weight']
max_value = max(values)
for i, row in m2_nlsp.iterrows():
    value = values[i]
    sim_geo = gpd.GeoSeries(row['geometry'])
    geo_j = sim_geo.to_json()

    color = plt.cm.Reds(value/max_value)
    color = mpl.colors.to_hex(color)

    if value == 0:
        geo_j = folium.GeoJson(data=geo_j,
                               style_function=lambda feature, color=color: {
                                   'fillColor': color,
                                   'color': "black",
                                   'weight': 0.5,
                                   'dashArray': '5, 5',
                                   'fillOpacity': 0,
                               })
    else:
        geo_j = folium.GeoJson(data=geo_j,
                               style_function=lambda feature, color=color: {
                                   'fillColor': color,
                                   'color': "black",
                                   'weight': 0.5,
                                   'dashArray': '5, 5',
                                   'fillOpacity': 0.8,
                               })

    folium.Popup(f'{value:.3f}').add_to(geo_j)
    geo_j.add_to(m4)
colormap = branca.colormap.linear.Reds_05.scale(0, max_value)
colormap = colormap.to_step(index=np.linspace(0, max_value, 1500))
colormap.caption = 'lbl × Area Ratio not covered by safe_delivery_box'
colormap.add_to(m4)
for lat, lon in zip(lats, lons):
    folium.Circle(location=[lat, lon], radius=radius,
                  color='black', fill_color='blue', weight=1).add_to(m4)
m4 #m4.save('./map/mclp.html')

In [ ]:
K = 10  # 설치할 갯수
M = cnt
opt_sites_org, f = mclp(m2_nlsp, K, M, radius)
print('커버되는 수요 지점의 개수 : {}'.format(f))

opt_df = pd.DataFrame(opt_sites_org, columns=['lon', 'lat'])
opt_df['geom'] = opt_df.apply(lambda r: Point(r['lon'], r['lat']), axis=1)
gdf = gpd.GeoDataFrame(opt_df, geometry='geom', crs='epsg:5179')
gdf = gdf.to_crs(epsg=4329)
gdf['lon'] = gdf['geom'].apply(lambda p: p.x)
gdf['lat'] = gdf['geom'].apply(lambda p: p.y)

m5 = m3
for lat, lon in zip(gdf['lat'], gdf['lon']):
    folium.Circle(location=[lat, lon], radius=radius,
                  color='black', fill_color='blue', weight=1).add_to(m5)
m5 #m5.save('./map/final.html')